In [45]:
# Core imports and helper utilities
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import time
import os
import pickle
import pathlib
import traceback

# Configuration
PROFILE_DIR = os.path.join(os.getcwd(), "chrome_profile")
HEADLESS = False  # Set to True if you need headless (may break some sites)
DEFAULT_TIMEOUT = 15


def setup_driver(profile_dir: str = PROFILE_DIR, headless: bool = HEADLESS, timeout: int = DEFAULT_TIMEOUT):
    """Start Chrome with a persistent profile and return (driver, wait).
    The profile dir keeps your logged-in session so you can log in once manually."""
    options = Options()
    options.add_argument(f"--user-data-dir={profile_dir}")
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--start-maximized")
    # Try to reduce automation flags
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_page_load_timeout(60)
    wait = WebDriverWait(driver, timeout)
    return driver, wait


def save_screenshot(driver, name: str = "error"):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"{name}_{ts}.png"
    try:
        driver.save_screenshot(out)
        print(f"⚠️ Screenshot saved: {out}")
    except Exception as e:
        print(f"⚠️ Failed to save screenshot: {e}")
    return out


def safe_click(driver, element, name: str = "", wait_after: float = 0.2):
    try:
        driver.execute_script("arguments[0].click();", element)
        time.sleep(wait_after)
        return True
    except Exception as e:
        print(f"✗ click failed ({name}): {e}")
        save_screenshot(driver, f"click_{name}")
        return False


def reservation_confirmed(driver, timeout: int = 5):
    """Heuristic to detect whether a reservation succeeded.
    Tries a few simple checks and returns True if any match.
    """
    try:
        # 1) Look for visible confirmation-like words on the page
        phrases = ['reserver', 'reservation', 'bevestig', 'confirmed', 'success']
        page_text = driver.page_source.lower()
        for p in phrases:
            if p in page_text:
                return True

        # 2) If the 'Create reservation' button is gone, assume success
        btns = driver.find_elements(By.XPATH, "//button//span[text()='Create reservation' or contains(., 'Create reservation') or contains(., 'Reservering')]")
        if not btns:
            return True
    except Exception:
        pass
    return False


def wait_for_login(driver, check_xpath: str = "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]", timeout: int = 120, poll_interval: float = 0.5):
    """Wait until the login tile is present or timeout expires.
    Returns True if login detected, False on timeout. Takes screenshots on timeout.

    This version checks immediately and then polls at `poll_interval` seconds.
    """
    end = time.time() + timeout
    print(f"Wachten op login (max {timeout}s)...")

    # immediate check
    try:
        elems = driver.find_elements(By.XPATH, check_xpath)
        if elems:
            print("✓ Login direct gedetecteerd")
            return True
    except Exception:
        pass

    next_print = time.time() + 5  # print status every ~5s to reduce noise
    while time.time() < end:
        try:
            elems = driver.find_elements(By.XPATH, check_xpath)
            if elems:
                print("✓ Login gedetecteerd")
                return True
        except Exception:
            pass
        if time.time() >= next_print:
            remaining = int(end - time.time())
            print(f"Wachten op login... nog {remaining}s", end='\r')
            next_print = time.time() + 5
        time.sleep(poll_interval)

    print("\n✗ Login niet gedetecteerd binnen timeout")
    try:
        save_screenshot(driver, "login_timeout")
    except Exception:
        pass
    return False


def attempt_reservation(driver, wait, a: int, b: int, datum_gezocht: str):
    try:
        print("=== BOOKING STARTEN ===")
        tile = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]")))
        tile.click()
        time.sleep(0.5)

        start_date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "mat-select[name='selectedStartDate']")))
        safe_click(driver, start_date, "start_date")
        print("✓ Start Date dropdown geopend")

        datum_option = wait.until(EC.element_to_be_clickable((By.XPATH, f"//mat-option[.//span[contains(text(), '{datum_gezocht}')]]")))
        safe_click(driver, datum_option, "datum_option")
        print("✓ Datum geselecteerd")

        ok_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button//span[text()='OK']")))
        safe_click(driver, ok_button, "OK")
        print("✓ OK geklikt")

        select_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Select']]")))
        safe_click(driver, select_button, "Select")
        print("✓ Select geklikt")

        select_start_time = wait.until(EC.element_to_be_clickable((By.XPATH, "//mat-select[@name='selectedStartTime']")))
        safe_click(driver, select_start_time, "select_start_time")
        xpath_b_uur = f"//mat-option//span[normalize-space()='{a}:00']"
        select_b_uur = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_b_uur)))
        safe_click(driver, select_b_uur, "start_time")
        print(f"✓ {a}:00 geselecteerd")

        select_end_time = wait.until(EC.element_to_be_clickable((By.XPATH, "//mat-select[@name='selectedEndTime']")))
        safe_click(driver, select_end_time, "select_end_time")
        xpath_e_uur = f"//mat-option//span[contains(text(), '{b}:00')]"
        
        select_e_uur = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_e_uur)))
        safe_click(driver, select_e_uur, "end_time")
        print(f"✓ {b}:00 geselecteerd")

        select_agree = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='mdc-checkbox']")))
        safe_click(driver, select_agree, "checkbox")
        
        make_reservation = wait.until(EC.element_to_be_clickable((By.XPATH, "//button//span[text()='Create reservation']")))
        safe_click(driver, make_reservation, "make_reservation")
        print("✓ Reservering aangevraagd")
        return True
    except Exception as e:
        print(f"✗ attempt_reservation failed: {e}")
        save_screenshot(driver, "attempt_failure")
        traceback.print_exc()
        return False

In [ ]:
# Config & scheduling
a = 9
b = 22
c = b - a
STARTTIJD = "18:00:00"  # Pas aan indien nodig
MAX_POGINGEN = 400
LOGIN_TIMEOUT = 400  # seconds to wait for login automatically (shorter default)
LOGIN_POLL_INTERVAL = 1
for_real = False 


In [47]:
# Setup driver with persistent profile
driver, wait = setup_driver()
print(f"✓ Chrome opened (profile: {PROFILE_DIR})")

# GA NAAR HOOFDPAGINA
driver.get("https://kurt3.ghum.kuleuven.be")
print("✓ Website geladen")

# Geef de sessie even tijd om te herstellen
time.sleep(3)

# CHECK OF JE INGELOGD BENT door te zoeken naar de tile
try:
    tile_ = driver.find_element(By.XPATH, "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]")
    print("✓ Je bent ingelogd!\n")
except Exception:
    print("✗ Niet ingelogd of sessie niet beschikbaar - log in in de geopende browser (profile).")
    input("Log in en druk ENTER om verder te gaan...")
    time.sleep(1)
    try:
        tile_ = driver.find_element(By.XPATH, "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]")
        print("✓ Je bent nu ingelogd!\n")
    except Exception:
        print("✗ Nog steeds niet ingelogd. Controleer het profiel of herhaal handmatig.")

✓ Chrome opened (profile: c:\Users\wille\Downloads\chrome_profile)
✓ Website geladen
✗ Niet ingelogd of sessie niet beschikbaar - log in in de geopende browser (profile).
✓ Je bent nu ingelogd!



In [50]:

while True:
    nu = datetime.now().strftime("%H:%M:%S")
    print(f"Huidige tijd: {nu}", end='\r')
    
    if nu >= STARTTIJD:
        print(f"\n✓ {STARTTIJD} bereikt! Starten...\n")
        break
    
    time.sleep(0.01)

driver.refresh()

Huidige tijd: 17:59:59
✓ 17:59:59 bereikt! Starten...



In [51]:
# Boekingslogica met retries
print("=== RESERVERING PROCES ===\n")
datum_over_week = datetime.now() + timedelta(days=8)
datum_gezocht = datum_over_week.strftime("%a %b %#d") 
print(f"Datum gezocht: {datum_gezocht}")

for attempt in range(1, MAX_POGINGEN + 1):
    print(f"\n--- Poging {attempt}/{MAX_POGINGEN} ---")
    ok = attempt_reservation(driver, wait, a, b, datum_gezocht)
    if ok:
        time.sleep(2)
        if reservation_confirmed(driver, timeout=5):
            print("✅ Reservering bevestigd!")
            break
        else:
            print("✗ Geen bevestiging gevonden, opnieuw proberen...")
    else:
        print("✗ Poging mislukt, opnieuw proberen...")
    time.sleep(0.01)
else:
    print("⚠️ Max pogingen bereikt zonder bevestiging")

print("\n✓✓✓ KLAAR!")



=== RESERVERING PROCES ===

Datum gezocht: Sat Jan 3

--- Poging 1/400 ---
=== BOOKING STARTEN ===
✓ Start Date dropdown geopend
✓ Datum geselecteerd
✓ OK geklikt
✓ Select geklikt
✓ 9:00 geselecteerd
✗ attempt_reservation failed: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561

⚠️ Screenshot saved: attempt_failure_20251226_180019.png
✗ Poging mislukt, opnieuw proberen...

--- Poging 2/400 ---
=== BOOKING STARTEN ===


Traceback (most recent call last):
  File "C:\Users\wille\AppData\Local\Temp\ipykernel_48980\3025936021.py", line 159, in attempt_reservation
    select_e_uur = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_e_uur)))
  File "c:\Users\wille\anaconda3\envs\tf_env\lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561



✗ attempt_reservation failed: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561

⚠️ Screenshot saved: attempt_failure_20251226_180035.png
✗ Poging mislukt, opnieuw proberen...

--- Poging 3/400 ---
=== BOOKING STARTEN ===


Traceback (most recent call last):
  File "C:\Users\wille\AppData\Local\Temp\ipykernel_48980\3025936021.py", line 128, in attempt_reservation
    tile = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]")))
  File "c:\Users\wille\anaconda3\envs\tf_env\lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561



✗ attempt_reservation failed: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561

⚠️ Screenshot saved: attempt_failure_20251226_180050.png
✗ Poging mislukt, opnieuw proberen...

--- Poging 4/400 ---
=== BOOKING STARTEN ===


...

✗ attempt_reservation failed: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561

⚠️ Screenshot saved: attempt_failure_20251226_180425.png
✗ Poging mislukt, opnieuw proberen...

--- Poging 18/400 ---
=== BOOKING STARTEN ===


Traceback (most recent call last):
  File "C:\Users\wille\AppData\Local\Temp\ipykernel_48980\3025936021.py", line 128, in attempt_reservation
    tile = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]")))
  File "c:\Users\wille\anaconda3\envs\tf_env\lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561



✗ attempt_reservation failed: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561

⚠️ Screenshot saved: attempt_failure_20251226_180440.png
✗ Poging mislukt, opnieuw proberen...

--- Poging 19/400 ---
=== BOOKING STARTEN ===


Traceback (most recent call last):
  File "C:\Users\wille\AppData\Local\Temp\ipykernel_48980\3025936021.py", line 128, in attempt_reservation
    tile = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'tile') and contains(., 'Agora - Silent Study Seat 144')]")))
  File "c:\Users\wille\anaconda3\envs\tf_env\lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3f12d3
	0x3f1314
	0x1de6dd
	0x2293a5
	0x22977b
	0x270382
	0x24b534
	0x26db13
	0x24b2e6
	0x21d321
	0x21e1d4
	0x645314
	0x6408cb
	0x65d1aa
	0x40b1d8
	0x4131dd
	0x3f95d8
	0x3f9799
	0x3e3b28
	0x75a85d49
	0x7708d5db
	0x7708d561



KeyboardInterrupt: 

In [ ]:
input("Druk ENTER om te sluiten...")
driver.quit()

# Diagnostics

In [ ]:
"""# Test of driver nog werkt
try:
    driver.current_url
    print("✓ Driver is actief")
except Exception:
    print("✗ Driver is gesloten - herstart nodig")
    # Herstart driver met persistent profiel
    driver, wait = setup_driver()
    driver.get("https://kurt3.ghum.kuleuven.be")
    print("✓ Driver herstart, controleren op login (automatisch)")
    if wait_for_login(driver, timeout=LOGIN_TIMEOUT, poll_interval=LOGIN_POLL_INTERVAL):
        print("✓ Driver herstart en ingelogd")
    else:
        print("✗ Nog steeds niet ingelogd na herstart; controleer de browser handmatig.")"""

✓ Driver is actief


In [ ]:
if diagnostics:
    # Diagnostic: try start Chrome with persistent profile + safe flags, fallback to temporary profile
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.chrome.options import Options
    import traceback

    flags = ["--no-sandbox", "--disable-dev-shm-usage", "--disable-extensions", "--no-first-run"]
    service = Service(ChromeDriverManager().install())

    print(f"Testing persistent profile at: {PROFILE_DIR}")
    opts = Options()
    for f in flags:
        opts.add_argument(f)
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    try:
        d = webdriver.Chrome(service=service, options=opts)
        print("✅ Started Chrome with persistent profile OK")
        d.quit()
    except Exception as e:
        print("❌ Failed to start with persistent profile:")
        traceback.print_exc()
        # Try temporary profile
        tmp_profile = os.path.join(os.getcwd(), "chrome_profile_test")
        print(f"Trying temporary profile at: {tmp_profile}")
        opts2 = Options()
        for f in flags:
            opts2.add_argument(f)
        opts2.add_argument(f"--user-data-dir={tmp_profile}")
        try:
            d2 = webdriver.Chrome(service=service, options=opts2)
            print("✅ Started Chrome with temporary profile — please log in in the opened browser window.")
            print("Once logged in, you can rerun the setup cell to let the script detect your session automatically.")
            # keep the browser open briefly so user can see it
            time.sleep(1)
            d2.quit()
        except Exception as e2:
            print("❌ Failed to start with temporary profile too:")
            traceback.print_exc()
else:
    print("einde")

NameError: name 'diagnostics' is not defined

In [ ]:
# Diagnostic: try minimal Chrome start to capture errors
if diagnostics:    
    try:
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        from selenium import webdriver
        svc = Service(ChromeDriverManager().install())
        opts = webdriver.ChromeOptions()
        opts.add_argument("--no-sandbox")
        opts.add_argument("--disable-dev-shm-usage")
        driver_test = webdriver.Chrome(service=svc, options=opts)
        print("Chrome started OK (minimal)")
        driver_test.quit()
    except Exception as e:
        import traceback
        print("❌ Failed to start minimal Chrome:")
        traceback.print_exc()


Chrome started OK (minimal)


In [ ]:
if diagnostics: # Quick diagnostic: environment, package versions, chromedriver path, running chrome processes
    import sys, os, traceback, subprocess
    print("Python:", sys.executable)
    print("Version:", sys.version)

    # package versions
    try:
        import selenium
        print("selenium:", selenium.__version__)
    except Exception as e:
        print("selenium: not installed or import failed", e)

    try:
        import webdriver_manager
        from webdriver_manager.chrome import ChromeDriverManager
        print("webdriver_manager:", webdriver_manager.__version__)
        try:
            path = ChromeDriverManager().install()
            print("chromedriver path:", path)
        except Exception as e:
            print("Could not install/find chromedriver:", e)
            traceback.print_exc()
    except Exception as e:
        print("webdriver_manager: not installed or import failed", e)

    # show chromedriver logs if present
    for fname in ("chromedriver_verbose.log", "chromedriver.log"):
        if os.path.exists(fname):
            print(f"\n--- tail of {fname} ---")
            with open(fname, "rb") as f:
                f.seek(max(0, os.path.getsize(fname)-8192))
                print(f.read().decode('utf-8', errors='replace'))

    # list running chrome/chromedriver processes (Windows tasklist)
    try:
        out = subprocess.check_output(["tasklist"], universal_newlines=True)
        lines = [L for L in out.splitlines() if "chrome.exe" in L.lower() or "chromedriver.exe" in L.lower()]
        print("\nActive chrome/chromedriver processes:")
        if lines:
            for l in lines:
                print(l)
        else:
            print("None")
    except Exception as e:
        print("Failed to query tasklist:", e)

    print("\nDiagnostic complete.")
else:
    print("einde")

einde


In [ ]:
# Diagnostic: capture current URL, title, visible tiles, screenshot and page-source snippet
if diagnostics:    
    try:
        print("Current URL:", driver.current_url)
        print("Title:", driver.title)

        tiles = driver.find_elements(By.XPATH, "//div[contains(@class, 'tile')]")
        print("Found tiles:", len(tiles))
        for i, t in enumerate(tiles[:10]):
            try:
                txt = t.text.strip().replace('\n', ' ')[:300]
                print(f"  {i+1}. {txt}")
            except Exception:
                print(f"  {i+1}. (no text)")

        ss = save_screenshot(driver, "diagnostic_page")
        print("Saved screenshot:", ss)

        ps = driver.page_source
        print('\n--- Page source snippet (first 2000 chars) ---')
        print(ps[:2000])
    except Exception as e:
        import traceback
        print("Diagnostic failed:", e)
        traceback.print_exc()
        try:
            save_screenshot(driver, "diagnostic_failure")
        except Exception:
            pass

Diagnostic failed: HTTPConnectionPool(host='localhost', port=57165): Max retries exceeded with url: /session/f4db788a87c64e1d4e15ec829bfaa581/url (Caused by NewConnectionError("HTTPConnection(host='localhost', port=57165): Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it"))


Traceback (most recent call last):
  File "c:\Users\wille\Downloads\.venv\Lib\site-packages\urllib3\connection.py", line 204, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wille\Downloads\.venv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\wille\Downloads\.venv\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\wille\Downloads\.venv\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wille\Downloads\.venv\Lib\site-packages\urllib3\connectionpool.py", line 493, in _make_request
  

⚠️ Failed to save screenshot: HTTPConnectionPool(host='localhost', port=57165): Max retries exceeded with url: /session/f4db788a87c64e1d4e15ec829bfaa581/screenshot (Caused by NewConnectionError("HTTPConnection(host='localhost', port=57165): Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it"))
